In [1]:
import datetime as dt
import os
import pandas as pd
import pandas_datareader.data as web
import bs4 as bs
import pickle
import requests
import numpy as np

In [2]:
#scraping current S&P 500 tickers from wikipedia.
def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text)
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        #below gets rid of unwanted ""\n"
        ticker = row.find('td').text.replace('\n','')
        #below changes "." to "-" to match yahoo's format for class-b tickers 
        if "." in ticker:
            ticker = ticker.replace('.','-')
            print('ticker replaced to', ticker)
        tickers.append(ticker)
    #below saves each ticker to a pickle file    
    with open("sp500tickers.pickle",'wb') as f:
        pickle.dump(tickers, f)
        
    print(tickers)
    
    return tickers

save_sp500_tickers()

ticker replaced to BRK-B
ticker replaced to BF-B
['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AMCR', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ANET', 'AJG', 'AIZ', 'T', 'ATO', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BKR', 'BLL', 'BAC', 'BK', 'BAX', 'BDX', 'BRK-B', 'BBY', 'BIO', 'BIIB', 'BLK', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BMY', 'AVGO', 'BR', 'BF-B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CARR', 'CTLT', 'CAT', 'CBOE', 'CBRE', 'CDW', 'CE', 'CNC', 'CNP', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'CTVA', 'COST', 'CCI', 'CSX

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AMCR',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'ADI',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ATO',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BKR',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BDX',
 'BRK-B',
 'BBY',
 'BIO',
 'BIIB',
 'BLK',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BMY',
 'AVGO',
 'BR',
 'BF-B',
 'CHRW',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CARR',
 'CTLT',
 'CAT',
 'CBOE',
 'CBRE',
 'CDW',
 'CE',
 'CNC',
 'CNP',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME',
 'CMS',
 'KO',
 'CTSH',
 'CL',
 'CMCSA',
 'CMA',
 'CAG',
 'CXO

In [3]:
#web scrape of yahoo finance data
def get_data_from_yahoo(reload_sp500=False):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle",'rb') as f:
            tickers = pickle.load(f)
    #creating a path to save data        
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')
    #start and end date for data from each ticker    
    start = dt.datetime(2000,1,1)
    end = dt.datetime(2020,10,27)
    #saving data for each ticker to its own file
    for ticker in tickers:
        print(ticker)
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, end)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))
            
get_data_from_yahoo()

MMM
ABT
ABBV
ABMD
ACN
ATVI
ADBE
AMD
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALXN
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
ANTM
AON
AOS
APA
AIV
AAPL
AMAT
APTV
ADM
ANET
AJG
AIZ
T
ATO
ADSK
ADP
AZO
AVB
AVY
BKR
BLL
BAC
BK
BAX
BDX
BRK-B
BBY
BIO
BIIB
BLK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BF-B
CHRW
COG
CDNS
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CERN
CF
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CTXS
CLX
CME
CMS
KO
CTSH
CL
CMCSA
CMA
CAG
CXO
COP
ED
STZ
COO
CPRT
GLW
CTVA
COST
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISCA
DISCK
DISH
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DRE
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
EMR
ETR
EOG
EFX
EQIX
EQR
ESS
EL
ETSY
EVRG
ES
RE
EXC
EXPE
EXPD
EXR
XOM
FFIV
FB
FAST
FRT
FDX
FIS
FITB
FE
FRC
FISV
FLT
FLIR
FLS
FMC
F
FTNT
FTV
FBHS
FOXA
FOX
BEN
FCX
GPS
GRMN
IT
GD
GE
GIS
GM
GPC
GILD
GL
GPN
GS
GWW
HAL
HBI
HIG
HAS
HCA
PEAK
HSIC
HSY
HES
HPE
HLT
HFC
HOLX
HD
HON
HRL
HST
HWM
HPQ
HUM
HBAN
HII

In [4]:
#concat for all tickers
import glob, os
files = glob.glob('stock_dfs/*.csv')
df = pd.concat([pd.read_csv(fp).assign(Ticker=os.path.basename(fp)) for fp in files])
df['Ticker'] = df['Ticker'].str.replace('.csv', '')

In [5]:
df['Date'] = pd.to_datetime(df['Date'])

In [6]:
df = df.reset_index(drop=True)
df

,Date,High,Low,Open,Close,Volume,Adj Close,Ticker
0,2000-01-03,55.125000,51.781250,54.96875,54.031250,53076000.0,40.733959,CSCO
1,2000-01-04,53.500000,50.875000,52.75000,51.000000,50805600.0,38.448730,CSCO
2,2000-01-05,52.250000,48.625000,50.03125,50.843750,68524000.0,38.330936,CSCO
3,2000-01-06,50.937500,49.312500,50.40625,50.000000,48242600.0,37.694832,CSCO
4,2000-01-07,53.000000,49.687500,49.68750,52.937500,62260600.0,39.909409,CSCO
...,...,...,...,...,...,...,...,...
2358222,2020-10-21,28.190001,27.750000,27.99000,27.900000,4018900.0,27.900000,PPL
2358223,2020-10-22,28.450001,27.500000,27.83000,28.430000,3912200.0,28.430000,PPL
2358224,2020-10-23,28.760000,28.280001,28.68000,28.670000,3360700.0,28.670000,PPL
2358225,2020-10-26,28.930000,28.280001,28.35000,28.920000,4670800.0,28.920000,PPL


In [7]:
df = df.rename(columns={"Adj Close": "adj_close"})
df

,Date,High,Low,Open,Close,Volume,adj_close,Ticker
0,2000-01-03,55.125000,51.781250,54.96875,54.031250,53076000.0,40.733959,CSCO
1,2000-01-04,53.500000,50.875000,52.75000,51.000000,50805600.0,38.448730,CSCO
2,2000-01-05,52.250000,48.625000,50.03125,50.843750,68524000.0,38.330936,CSCO
3,2000-01-06,50.937500,49.312500,50.40625,50.000000,48242600.0,37.694832,CSCO
4,2000-01-07,53.000000,49.687500,49.68750,52.937500,62260600.0,39.909409,CSCO
...,...,...,...,...,...,...,...,...
2358222,2020-10-21,28.190001,27.750000,27.99000,27.900000,4018900.0,27.900000,PPL
2358223,2020-10-22,28.450001,27.500000,27.83000,28.430000,3912200.0,28.430000,PPL
2358224,2020-10-23,28.760000,28.280001,28.68000,28.670000,3360700.0,28.670000,PPL
2358225,2020-10-26,28.930000,28.280001,28.35000,28.920000,4670800.0,28.920000,PPL


In [8]:
#initial feature engineering and creating the target (twenty_one_day_performance)
df['daily_performance'] = df.groupby('Ticker').adj_close.pct_change().fillna(0)
df['twenty_one_day_performance'] = df.groupby('Ticker').adj_close.pct_change(periods=21).fillna(0)
df.loc[df['twenty_one_day_performance'] >= 0.06, 'one_month_6_pct_gain'] = "Yes"
df.loc[df['twenty_one_day_performance'] < 0.06, 'one_month_6_pct_gain'] = "No"

In [9]:
df.loc[8930:8960]

,Date,High,Low,Open,Close,Volume,adj_close,Ticker,daily_performance,twenty_one_day_performance,one_month_6_pct_gain
8930,2020-10-05,36.580002,35.419998,36.430000,36.200001,18046400.0,36.200001,UAL,0.005276,-0.032086,No
8931,2020-10-06,37.090000,34.840000,36.730000,34.880001,28719000.0,34.880001,UAL,-0.036464,-0.087150,No
8932,2020-10-07,37.220001,35.880001,36.509998,36.380001,27677300.0,36.380001,UAL,0.043005,-0.024142,No
8933,2020-10-08,37.689999,35.849998,37.160000,37.000000,28333500.0,37.000000,UAL,0.017042,0.026922,No
8934,2020-10-09,37.799999,36.270000,37.049999,37.119999,25311800.0,37.119999,UAL,0.003243,0.018381,No
8935,2020-10-12,36.990002,36.259998,36.700001,36.380001,20202600.0,36.380001,UAL,-0.019935,0.008594,No
8936,2020-10-13,35.950001,35.220001,35.669998,35.259998,17303000.0,35.259998,UAL,-0.030786,-0.040283,No
8937,2020-10-14,36.330002,35.169998,35.259998,35.610001,15064400.0,35.610001,UAL,0.009926,-0.032337,No
8938,2020-10-15,35.380001,33.639999,35.180000,34.250000,23652000.0,34.250000,UAL,-0.038192,-0.105744,No
8939,2020-10-16,34.580002,33.820000,34.310001,34.160000,16210000.0,34.160000,UAL,-0.002628,-0.109257,No


In [10]:
#replacing ticker to be more useful with pandas and python syntax
df["Ticker"]= df["Ticker"].str.replace('BRK-B', 'BRK_B')
df["Ticker"]= df["Ticker"].str.replace('BF-B', 'BF_B')

In [11]:
df.Ticker.unique()

array(['CSCO', 'UAL', 'TROW', 'ISRG', 'NVR', 'PRGO', 'TPR', 'DVN', 'CE',
       'MRO', 'BA', 'VRTX', 'GILD', 'NLSN', 'EQIX', 'TER', 'TIF', 'MDT',
       'V', 'QRVO', 'A', 'FOX', 'FLT', 'MO', 'SWKS', 'MCHP', 'CDNS',
       'WLTW', 'MSCI', 'CHTR', 'EIX', 'BBY', 'WBA', 'LVS', 'HCA', 'AJG',
       'DTE', 'C', 'T', 'CF', 'DISH', 'MGM', 'HUM', 'CBOE', 'CFG', 'WU',
       'APH', 'SYY', 'MSI', 'FCX', 'ADM', 'LH', 'PKI', 'LNT', 'BAC',
       'LNC', 'PSX', 'GPN', 'PPG', 'IRM', 'IQV', 'ESS', 'NOV', 'HAL',
       'STZ', 'FLS', 'DXC', 'ADI', 'F', 'ADBE', 'CPRT', 'TDG', 'TFX',
       'ULTA', 'ARE', 'SYK', 'CB', 'TSN', 'PEP', 'PEG', 'NOW', 'LLY',
       'COST', 'REG', 'NWS', 'LOW', 'MDLZ', 'BKNG', 'ZBRA', 'FMC', 'XEL',
       'AIZ', 'CERN', 'MET', 'FTV', 'DLR', 'XRAY', 'FAST', 'TJX', 'SNA',
       'MPC', 'BR', 'D', 'MRK', 'STX', 'NOC', 'BXP', 'KHC', 'IPG', 'UNP',
       'ALLE', 'ABBV', 'ORCL', 'ECL', 'ETR', 'EBAY', 'SBUX', 'IR', 'AMT',
       'INTU', 'DPZ', 'PAYC', 'DRE', 'CMA', 'IPGP', 'PG', 'CAT', 

In [12]:
#creating an array of tickers
unique_tickers = df.Ticker.unique()
#creating a dictionary to be used in a for loop
companydict = {elem : pd.DataFrame() for elem in unique_tickers}
for key in companydict.keys():
    companydict[key] = df[:][df.Ticker == key]
    #creating the 5 day performance column that preceeds the 21 day prediction period
    preceeding_5_day_perf = (companydict[key]['adj_close'].shift(21) - companydict[key]['adj_close'].shift(26)) / companydict[key]['adj_close'].shift(26)
    companydict[key] = companydict[key].assign(preceeding_5_day_perf=preceeding_5_day_perf).fillna(0)
    
    preceeding_16_day_perf_1M = (companydict[key]['adj_close'].shift(26) - companydict[key]['adj_close'].shift(42)) / companydict[key]['adj_close'].shift(42)
    companydict[key] = companydict[key].assign(preceeding_16_day_perf_1M=preceeding_16_day_perf_1M).fillna(0)
    
    preceeding_2_month_perf_3M = (companydict[key]['adj_close'].shift(42) - companydict[key]['adj_close'].shift(84)) / companydict[key]['adj_close'].shift(84)
    companydict[key] = companydict[key].assign(preceeding_2_month_perf_3M=preceeding_2_month_perf_3M).fillna(0)
    
    preceeding_3_month_perf_6M = (companydict[key]['adj_close'].shift(84) - companydict[key]['adj_close'].shift(147)) / companydict[key]['adj_close'].shift(147)
    companydict[key] = companydict[key].assign(preceeding_3_month_perf_6M=preceeding_3_month_perf_6M).fillna(0)
    
    preceeding_6_month_perf_1Y = (companydict[key]['adj_close'].shift(147) - companydict[key]['adj_close'].shift(273)) / companydict[key]['adj_close'].shift(273)
    companydict[key] = companydict[key].assign(preceeding_6_month_perf_1Y=preceeding_6_month_perf_1Y).fillna(0)
    
    preceeding_2_year_perf_3Y = (companydict[key]['adj_close'].shift(273) - companydict[key]['adj_close'].shift(777)) / companydict[key]['adj_close'].shift(777)
    companydict[key] = companydict[key].assign(preceeding_2_year_perf_3Y=preceeding_2_year_perf_3Y).fillna(0)
    
    preceeding_2_year_perf_5Y = (companydict[key]['adj_close'].shift(777) - companydict[key]['adj_close'].shift(1281)) / companydict[key]['adj_close'].shift(1281)
    companydict[key] = companydict[key].assign(preceeding_2_year_perf_5Y=preceeding_2_year_perf_5Y).fillna(0)
    
    preceeding_5_year_perf_10Y = (companydict[key]['adj_close'].shift(1281) - companydict[key]['adj_close'].shift(2541)) / companydict[key]['adj_close'].shift(2541)
    companydict[key] = companydict[key].assign(preceeding_5_year_perf_10Y=preceeding_5_year_perf_10Y).fillna(0)

    #creating a new df for each ticker (eg. CSCO ticker will be in df_CSCO)
    globals()['df_{}'.format(key)] = companydict[key]

In [13]:
df_list = [df_CSCO, df_UAL, df_TROW, df_ISRG, df_NVR, df_PRGO, df_TPR, df_DVN, df_CE, df_MRO, df_BA, df_VRTX, df_GILD, df_NLSN, df_EQIX, df_TER, df_TIF, df_MDT, df_V, df_QRVO, df_A, df_FOX, df_FLT, df_MO, df_SWKS, df_MCHP, df_CDNS, df_WLTW, df_MSCI, df_CHTR, df_EIX, df_BBY, df_WBA, df_LVS, df_HCA, df_AJG, df_DTE, df_C, df_T, df_CF, df_DISH, df_MGM, df_HUM, df_CBOE, df_CFG, df_WU, df_APH, df_SYY, df_MSI, df_FCX, df_ADM, df_LH, df_PKI, df_LNT, df_BAC, df_LNC, df_PSX, df_GPN, df_PPG, df_IRM, df_IQV, df_ESS, df_NOV, df_HAL, df_STZ, df_FLS, df_DXC, df_ADI, df_F, df_ADBE, df_CPRT, df_TDG, df_TFX, df_ULTA, df_ARE, df_SYK, df_CB, df_TSN, df_PEP, df_PEG, df_NOW, df_LLY, df_COST, df_REG, df_NWS, df_LOW, df_MDLZ, df_BKNG, df_ZBRA, df_FMC, df_XEL, df_AIZ, df_CERN, df_MET, df_FTV, df_DLR, df_XRAY, df_FAST, df_TJX, df_SNA, df_MPC, df_BR, df_D, df_MRK, df_STX, df_NOC, df_BXP, df_KHC, df_IPG, df_UNP, df_ALLE, df_ABBV, df_ORCL, df_ECL, df_ETR, df_EBAY, df_SBUX, df_IR, df_AMT, df_INTU, df_DPZ, df_PAYC, df_DRE, df_CMA, df_IPGP, df_PG, df_CAT, df_ODFL, df_MCD, df_MNST, df_AMZN, df_INTC, df_VNT, df_PNR, df_GLW, df_BDX, df_KMI, df_PWR, df_APTV, df_DXCM, df_EXR, df_WELL, df_HOLX, df_EXPD, df_GM, df_TXN, df_VRSK, df_SJM, df_TMO, df_OXY, df_RL, df_CCI, df_MMM, df_MOS, df_FTNT, df_HSY, df_JNPR, df_DHI, df_ED, df_ES, df_ADSK, df_GL, df_IP, df_EXPE, df_KO, df_PCAR, df_WDC, df_LUMN, df_PYPL, df_NEE, df_UPS, df_FLIR, df_LEG, df_EMR, df_MSFT, df_ANSS, df_CTAS, df_BIO, df_UDR, df_CTLT, df_WEC, df_AME, df_IT, df_DD, df_ACN, df_VRSN, df_EW, df_CMG, df_AWK, df_COO, df_SHW, df_HPQ, df_AMAT, df_CCL, df_MLM, df_AVY, df_AAP, df_ATVI, df_EVRG, df_EA, df_DE, df_SPG, df_AMD, df_MYL, df_KLAC, df_NDAQ, df_URI, df_WHR, df_RTX, df_PNC, df_KMX, df_WRK, df_BIIB, df_NVDA, df_CHRW, df_ROP, df_IDXX, df_EXC, df_HES, df_HD, df_ALB, df_VLO, df_AON, df_ZTS, df_FDX, df_DG, df_TYL, df_HIG, df_CMS, df_CAG, df_INCY, df_SCHW, df_HSIC, df_AZO, df_AXP, df_HPE, df_DFS, df_SEE, df_HRL, df_SO, df_FRT, df_ZBH, df_FRC, df_CME, df_XOM, df_AMP, df_CVX, df_CMCSA, df_PNW, df_ICE, df_CTXS, df_BEN, df_DISCK, df_UHS, df_BKR, df_EMN, df_SBAC, df_ROK, df_NRG, df_NSC, df_NKE, df_FIS, df_FANG, df_VTR, df_MAS, df_RF, df_ETSY, df_AMCR, df_TAP, df_MAR, df_XYL, df_CMI, df_FB, df_MTD, df_VAR, df_BF_B, df_NLOK, df_KR, df_PLD, df_IBM, df_USB, df_BSX, df_LKQ, df_FBHS, df_LIN, df_ITW, df_TWTR, df_EOG, df_PVH, df_KMB, df_PEAK, df_SPGI, df_NEM, df_WFC, df_CTVA, df_EL, df_GS, df_GD, df_CNP, df_PM, df_RE, df_MCO, df_CLX, df_CAH, df_DGX, df_AVB, df_DIS, df_CBRE, df_GE, df_HII, df_LDOS, df_ALL, df_ETN, df_ALGN, df_NFLX, df_LEN, df_FITB, df_WST, df_GWW, df_NTRS, df_CVS, df_AOS, df_FE, df_ABC, df_JPM, df_ABT, df_CXO, df_OMC, df_COF, df_TSCO, df_PH, df_HST, df_JBHT, df_ATO, df_COP, df_DHR, df_COG, df_CNC, df_MCK, df_TXT, df_MTB, df_HFC, df_DISCA, df_AKAM, df_ROL, df_RMD, df_WRB, df_GOOGL, df_ANET, df_PAYX, df_ALK, df_DRI, df_ILMN, df_AAL, df_XLNX, df_MAA, df_MMC, df_FOXA, df_POOL, df_FFIV, df_VNO, df_CINF, df_VMC, df_MKTX, df_SRE, df_LHX, df_ORLY, df_IVZ, df_RCL, df_PXD, df_SNPS, df_GOOG, df_SIVB, df_YUM, df_LYV, df_PFE, df_AIV, df_AVGO, df_DUK, df_REGN, df_CL, df_VFC, df_UA, df_VZ, df_JCI, df_AMGN, df_TEL, df_JKHY, df_ADP, df_LB, df_STT, df_RSG, df_IFF, df_ANTM, df_GPS, df_BLL, df_CARR, df_QCOM, df_LYB, df_GIS, df_PHM, df_ROST, df_LUV, df_ALXN, df_LW, df_MS, df_CPB, df_OKE, df_BK, df_J, df_SYF, df_CHD, df_SLG, df_HWM, df_MHK, df_TFC, df_INFO, df_DAL, df_APA, df_K, df_AFL, df_CSX, df_NI, df_PFG, df_NCLH, df_ZION, df_RJF, df_HBAN, df_UNH, df_PRU, df_GPC, df_FISV, df_WMB, df_EQR, df_MXIM, df_PBCT, df_KSU, df_DVA, df_AIG, df_MA, df_HBI, df_HON, df_O, df_NWSA, df_TTWO, df_AES, df_SLB, df_TT, df_XRX, df_TGT, df_AAPL, df_MKC, df_OTIS, df_TDY, df_WY, df_APD, df_GRMN, df_AEE, df_HLT, df_DLTR, df_STE, df_HAS, df_TMUS, df_WMT, df_NTAP, df_KIM, df_BAX, df_LMT, df_ABMD, df_KEY, df_UNM, df_KEYS, df_BMY, df_PSA, df_WYNN, df_RHI, df_EFX, df_NUE, df_PKG, df_WAB, df_CTSH, df_SWK, df_MU, df_TRV, df_BRK_B, df_L, df_AEP, df_CI, df_DOW, df_CDW, df_JNJ, df_WM, df_DOV, df_FTI, df_VIAC, df_CRM, df_PGR, df_WAT, df_IEX, df_BWA, df_LRCX, df_NWL, df_UAA, df_BLK, df_PPL]

In [14]:
df[df['Ticker']=='BF_B']

,Date,High,Low,Open,Close,Volume,adj_close,Ticker,daily_performance,twenty_one_day_performance,one_month_6_pct_gain
1269829,2000-01-03,6.200000,6.066667,6.080000,6.146667,1075000.0,1.052858,BF_B,0.000000,0.000000,No
1269830,2000-01-04,6.200000,5.973333,6.120000,6.033333,806200.0,1.033445,BF_B,-0.018438,0.000000,No
1269831,2000-01-05,6.133333,6.000000,6.000000,6.080000,744200.0,1.041439,BF_B,0.007735,0.000000,No
1269832,2000-01-06,6.146667,6.040000,6.053333,6.120000,1078000.0,1.048290,BF_B,0.006579,0.000000,No
1269833,2000-01-07,6.193333,6.106667,6.106667,6.186666,1163200.0,1.059709,BF_B,0.010893,0.000000,No
...,...,...,...,...,...,...,...,...,...,...,...
1275063,2020-10-21,75.180000,74.279999,74.279999,74.739998,713400.0,74.739998,BF_B,0.003626,-0.018516,No
1275064,2020-10-22,75.589996,74.690002,75.059998,75.180000,709000.0,75.180000,BF_B,0.005887,-0.002124,No
1275065,2020-10-23,76.059998,74.169998,75.730003,74.400002,624700.0,74.400002,BF_B,-0.010375,-0.005215,No
1275066,2020-10-26,73.900002,72.199997,73.769997,72.760002,694400.0,72.760002,BF_B,-0.022043,-0.023356,No


In [15]:
#creating a main df of all tickers
df_final = pd.concat(df_list)

In [16]:
df_final[df_final['Ticker']=='BF_B']

,Date,High,Low,Open,Close,Volume,adj_close,Ticker,daily_performance,twenty_one_day_performance,one_month_6_pct_gain,preceeding_5_day_perf,preceeding_16_day_perf_1M,preceeding_2_month_perf_3M,preceeding_3_month_perf_6M,preceeding_6_month_perf_1Y,preceeding_2_year_perf_3Y,preceeding_2_year_perf_5Y,preceeding_5_year_perf_10Y
1269829,2000-01-03,6.200000,6.066667,6.080000,6.146667,1075000.0,1.052858,BF_B,0.000000,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1269830,2000-01-04,6.200000,5.973333,6.120000,6.033333,806200.0,1.033445,BF_B,-0.018438,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1269831,2000-01-05,6.133333,6.000000,6.000000,6.080000,744200.0,1.041439,BF_B,0.007735,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1269832,2000-01-06,6.146667,6.040000,6.053333,6.120000,1078000.0,1.048290,BF_B,0.006579,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1269833,2000-01-07,6.193333,6.106667,6.106667,6.186666,1163200.0,1.059709,BF_B,0.010893,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1275063,2020-10-21,75.180000,74.279999,74.279999,74.739998,713400.0,74.739998,BF_B,0.003626,-0.018516,No,-0.035588,0.097841,0.106370,0.333955,-0.218961,0.525754,0.140439,2.141893
1275064,2020-10-22,75.589996,74.690002,75.059998,75.180000,709000.0,75.180000,BF_B,0.005887,-0.002124,No,-0.033359,0.072499,0.141357,0.228263,-0.171041,0.538489,0.141510,2.089499
1275065,2020-10-23,76.059998,74.169998,75.730003,74.400002,624700.0,74.400002,BF_B,-0.010375,-0.005215,No,-0.032471,0.068829,0.144301,0.148860,-0.115374,0.523195,0.153563,2.105068
1275066,2020-10-26,73.900002,72.199997,73.769997,72.760002,694400.0,72.760002,BF_B,-0.022043,-0.023356,No,-0.024358,0.051335,0.193279,0.122252,-0.126048,0.518084,0.154935,2.103347


In [17]:
df_final.to_pickle("./SP_500_df_10_28.pkl")

In [18]:
df_final = pd.read_pickle("./SP_500_df_10_28.pkl")
df_final

,Date,High,Low,Open,Close,Volume,adj_close,Ticker,daily_performance,twenty_one_day_performance,one_month_6_pct_gain,preceeding_5_day_perf,preceeding_16_day_perf_1M,preceeding_2_month_perf_3M,preceeding_3_month_perf_6M,preceeding_6_month_perf_1Y,preceeding_2_year_perf_3Y,preceeding_2_year_perf_5Y,preceeding_5_year_perf_10Y
0,2000-01-03,55.125000,51.781250,54.96875,54.031250,53076000.0,40.733959,CSCO,0.000000,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2000-01-04,53.500000,50.875000,52.75000,51.000000,50805600.0,38.448730,CSCO,-0.056101,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-01-05,52.250000,48.625000,50.03125,50.843750,68524000.0,38.330936,CSCO,-0.003064,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2000-01-06,50.937500,49.312500,50.40625,50.000000,48242600.0,37.694832,CSCO,-0.016595,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2000-01-07,53.000000,49.687500,49.68750,52.937500,62260600.0,39.909409,CSCO,0.058750,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358222,2020-10-21,28.190001,27.750000,27.99000,27.900000,4018900.0,27.900000,PPL,-0.006056,0.062452,Yes,-0.052670,0.003337,0.088509,0.229453,-0.310375,-0.092920,0.349575,0.604111
2358223,2020-10-22,28.450001,27.500000,27.83000,28.430000,3912200.0,28.430000,PPL,0.018996,0.097260,Yes,-0.074973,0.002041,0.111677,0.163755,-0.278003,-0.092507,0.361131,0.595228
2358224,2020-10-23,28.760000,28.280001,28.68000,28.670000,3360700.0,28.670000,PPL,0.008442,0.099732,Yes,-0.052000,-0.000347,0.091051,0.051530,-0.187695,-0.096952,0.347471,0.593098
2358225,2020-10-26,28.930000,28.280001,28.35000,28.920000,4670800.0,28.920000,PPL,0.008720,0.092558,Yes,-0.012313,-0.017346,0.104109,-0.000232,-0.177781,-0.084762,0.331997,0.597798


In [19]:
%pylab inline

# sets backend to render higher res images
%config InlineBackend.figure_formats = ['retina']

#######################
#       imports       #
#######################
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
import xgboost as xgb

sns.set_style("whitegrid")

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.cm as cm
import numpy as np
import random

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

Populating the interactive namespace from numpy and matplotlib


In [21]:
df = pd.read_pickle("./SP_500_df_10_28.pkl")

In [22]:
df

,Date,High,Low,Open,Close,Volume,adj_close,Ticker,daily_performance,twenty_one_day_performance,one_month_6_pct_gain,preceeding_5_day_perf,preceeding_16_day_perf_1M,preceeding_2_month_perf_3M,preceeding_3_month_perf_6M,preceeding_6_month_perf_1Y,preceeding_2_year_perf_3Y,preceeding_2_year_perf_5Y,preceeding_5_year_perf_10Y
0,2000-01-03,55.125000,51.781250,54.96875,54.031250,53076000.0,40.733959,CSCO,0.000000,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2000-01-04,53.500000,50.875000,52.75000,51.000000,50805600.0,38.448730,CSCO,-0.056101,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2000-01-05,52.250000,48.625000,50.03125,50.843750,68524000.0,38.330936,CSCO,-0.003064,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2000-01-06,50.937500,49.312500,50.40625,50.000000,48242600.0,37.694832,CSCO,-0.016595,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2000-01-07,53.000000,49.687500,49.68750,52.937500,62260600.0,39.909409,CSCO,0.058750,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358222,2020-10-21,28.190001,27.750000,27.99000,27.900000,4018900.0,27.900000,PPL,-0.006056,0.062452,Yes,-0.052670,0.003337,0.088509,0.229453,-0.310375,-0.092920,0.349575,0.604111
2358223,2020-10-22,28.450001,27.500000,27.83000,28.430000,3912200.0,28.430000,PPL,0.018996,0.097260,Yes,-0.074973,0.002041,0.111677,0.163755,-0.278003,-0.092507,0.361131,0.595228
2358224,2020-10-23,28.760000,28.280001,28.68000,28.670000,3360700.0,28.670000,PPL,0.008442,0.099732,Yes,-0.052000,-0.000347,0.091051,0.051530,-0.187695,-0.096952,0.347471,0.593098
2358225,2020-10-26,28.930000,28.280001,28.35000,28.920000,4670800.0,28.920000,PPL,0.008720,0.092558,Yes,-0.012313,-0.017346,0.104109,-0.000232,-0.177781,-0.084762,0.331997,0.597798


In [23]:
X = df[['preceeding_5_day_perf', 'preceeding_16_day_perf_1M', 'preceeding_2_month_perf_3M', 'preceeding_3_month_perf_6M', 'preceeding_6_month_perf_1Y', 'preceeding_2_year_perf_3Y']]
y = df['one_month_6_pct_gain']

In [24]:
import pickle as pkl

#refer to Call_option_screener_models_project_3 file to see how the below model was created
gbm = pkl.load(open("model_call_option_screener.pickle.dat", "rb"))

df['final_model_prediction']=gbm.predict(X)

df

,Date,High,Low,Open,Close,Volume,adj_close,Ticker,daily_performance,twenty_one_day_performance,one_month_6_pct_gain,preceeding_5_day_perf,preceeding_16_day_perf_1M,preceeding_2_month_perf_3M,preceeding_3_month_perf_6M,preceeding_6_month_perf_1Y,preceeding_2_year_perf_3Y,preceeding_2_year_perf_5Y,preceeding_5_year_perf_10Y,final_model_prediction
0,2000-01-03,55.125000,51.781250,54.96875,54.031250,53076000.0,40.733959,CSCO,0.000000,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
1,2000-01-04,53.500000,50.875000,52.75000,51.000000,50805600.0,38.448730,CSCO,-0.056101,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
2,2000-01-05,52.250000,48.625000,50.03125,50.843750,68524000.0,38.330936,CSCO,-0.003064,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
3,2000-01-06,50.937500,49.312500,50.40625,50.000000,48242600.0,37.694832,CSCO,-0.016595,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
4,2000-01-07,53.000000,49.687500,49.68750,52.937500,62260600.0,39.909409,CSCO,0.058750,0.000000,No,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2358222,2020-10-21,28.190001,27.750000,27.99000,27.900000,4018900.0,27.900000,PPL,-0.006056,0.062452,Yes,-0.052670,0.003337,0.088509,0.229453,-0.310375,-0.092920,0.349575,0.604111,0
2358223,2020-10-22,28.450001,27.500000,27.83000,28.430000,3912200.0,28.430000,PPL,0.018996,0.097260,Yes,-0.074973,0.002041,0.111677,0.163755,-0.278003,-0.092507,0.361131,0.595228,0
2358224,2020-10-23,28.760000,28.280001,28.68000,28.670000,3360700.0,28.670000,PPL,0.008442,0.099732,Yes,-0.052000,-0.000347,0.091051,0.051530,-0.187695,-0.096952,0.347471,0.593098,0
2358225,2020-10-26,28.930000,28.280001,28.35000,28.920000,4670800.0,28.920000,PPL,0.008720,0.092558,Yes,-0.012313,-0.017346,0.104109,-0.000232,-0.177781,-0.084762,0.331997,0.597798,0


In [25]:
import datetime 
df_10_27_2020 = df[df['Date']=="2020-10-27"]


df_10_27_2020_y = df_10_27_2020[df_10_27_2020['final_model_prediction']==1]

df_10_27_2020_y

,Date,High,Low,Open,Close,Volume,adj_close,Ticker,daily_performance,twenty_one_day_performance,one_month_6_pct_gain,preceeding_5_day_perf,preceeding_16_day_perf_1M,preceeding_2_month_perf_3M,preceeding_3_month_perf_6M,preceeding_6_month_perf_1Y,preceeding_2_year_perf_3Y,preceeding_2_year_perf_5Y,preceeding_5_year_perf_10Y,final_model_prediction
40072,2020-10-27,8.850000,8.600000,8.720000,8.690000,8520700.0,8.690000,DVN,-0.013621,-0.113265,No,0.057174,-0.127658,-0.011837,0.767114,-0.734291,-0.330078,-0.011934,-0.338305,1
464862,2020-10-27,29.000000,28.049999,28.440001,28.190001,4676000.0,28.190001,MPC,-0.014336,-0.058765,No,-0.041600,-0.106632,0.003190,0.479547,-0.603047,0.198981,0.247546,0.000000,1
935332,2020-10-27,82.370003,77.570000,82.000000,78.879997,156374200.0,78.879997,AMD,-0.040739,-0.007549,No,0.019759,-0.069928,0.666667,0.050564,0.666435,1.306827,6.280701,-0.732812,1
1219993,2020-10-27,27.440001,26.450001,27.350000,26.570000,4160900.0,26.570000,FANG,-0.034871,-0.156240,No,-0.018392,-0.193362,-0.031900,0.772516,-0.735619,-0.066236,0.471747,0.000000,1
1977586,2020-10-27,17.070000,16.510000,16.850000,16.530001,12921700.0,16.530001,NCLH,-0.016071,-0.008993,No,0.112742,-0.099700,0.005435,0.550562,-0.790670,-0.070505,-0.095419,0.000000,1
2085686,2020-10-27,15.230000,14.750000,15.110000,15.090000,14745700.0,15.090000,SLB,-0.005274,-0.082675,No,-0.047481,-0.070358,0.038312,0.365436,-0.601979,-0.458474,-0.003949,0.373880,1
2308223,2020-10-27,6.090000,5.700000,5.900000,5.700000,9214500.0,5.700000,FTI,-0.065574,-0.105181,No,-0.049254,-0.118421,0.117647,-0.035461,-0.700374,-0.060517,-0.124501,0.215596,1


In [26]:
df_10_27_2020_y_simple = df_10_27_2020_y[['Date', 
                                         'Ticker',
                                         'adj_close',
                                         'final_model_prediction']]

df_10_27_2020_y_simple

,Date,Ticker,adj_close,final_model_prediction
40072,2020-10-27,DVN,8.690000,1
464862,2020-10-27,MPC,28.190001,1
935332,2020-10-27,AMD,78.879997,1
1219993,2020-10-27,FANG,26.570000,1
1977586,2020-10-27,NCLH,16.530001,1
2085686,2020-10-27,SLB,15.090000,1
2308223,2020-10-27,FTI,5.700000,1


In [27]:
df_10_27_2020_y_simple['final_model_prediction'] = 'Yes, execute'

df_10_27_2020_y_simple.to_pickle("./yes_execute_10_27.pkl")

<ipython-input-27-623e1ef6523e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_10_27_2020_y_simple['final_model_prediction'] = 'Yes, execute'


In [29]:
df_10_27_2020_y_simple

,Date,Ticker,adj_close,final_model_prediction
40072,2020-10-27,DVN,8.690000,"Yes, execute"
464862,2020-10-27,MPC,28.190001,"Yes, execute"
935332,2020-10-27,AMD,78.879997,"Yes, execute"
1219993,2020-10-27,FANG,26.570000,"Yes, execute"
1977586,2020-10-27,NCLH,16.530001,"Yes, execute"
2085686,2020-10-27,SLB,15.090000,"Yes, execute"
2308223,2020-10-27,FTI,5.700000,"Yes, execute"
